In [19]:
import os

def find_project_root(start_path):
    current = start_path
    while True:
        if os.path.exists(os.path.join(current, ".git")):
            return current
        parent = os.path.dirname(current)
        if parent == current:
            raise RuntimeError("No se encontró la raíz del proyecto (.git).")
        current = parent

PROJECT_ROOT = find_project_root(os.getcwd())

DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")
DATA_INTERIM   = os.path.join(PROJECT_ROOT, "data", "interim")
DATA_RAW       = os.path.join(PROJECT_ROOT, "data", "raw")

DATASET_INPUT = os.path.join(DATA_PROCESSED, "dataset_integrado.parquet")
DATASET_OUTPUT = os.path.join(DATA_PROCESSED, "dataset_modelado.parquet")

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATASET_INPUT:", DATASET_INPUT)
print("DATASET_OUTPUT:", DATASET_OUTPUT)


PROJECT_ROOT: c:\IA_Investigacion\Deteccion_Corrupcion
DATASET_INPUT: c:\IA_Investigacion\Deteccion_Corrupcion\data\processed\dataset_integrado.parquet
DATASET_OUTPUT: c:\IA_Investigacion\Deteccion_Corrupcion\data\processed\dataset_modelado.parquet


1. Cargar el parquet ORIGINAL y crear COPIA

In [20]:
import pandas as pd

df_raw = pd.read_parquet(DATASET_INPUT)
df = df_raw.copy()

print("Filas y columnas:", df.shape)
df.head()


Filas y columnas: (14179, 56)


,CODIGO_UNICO,SECTOR,DEPARTAMENTO,NIVEL_GOBIERNO,IDENTIFICADOR_OBRA,PROCESO,OBJETO_PROCESO,CODIGO_OBRA,NOMBREOBRA,METODO_CONTRATACION,...,MES,PLANIFICADO,REAL,IND_INTERVENSION,IND_RESIDENTE,IND_MONTO_ADELANTO_MATERIALES,IND_MONTO_ADELANTO_DIRECTO,IND_FECHA_ADELANTO_MATERIALES,IND_FECHA_ADELANTO_DIRECTO,y_riesgo
0,2002060.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,2002060-2434-702592-2064311-19777,3.0,Consultoría de Obra,19777.0,Obra Carretera Puente Chino - Aguaytía; sector...,Concurso Público,...,NaN,NaN,NaN,None,None,None,None,None,None,0
1,2002210.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,2002210-8880-869397-2169938-826,60.0,Consultoría de Obra,826.0,Rehabilitación y Mejoramiento de la Carretera ...,Concurso Público,...,NaN,NaN,NaN,None,None,None,None,None,None,0
2,2002210.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,2002210-8880-870112-2169901-826,60.0,Consultoría de Obra,826.0,Rehabilitación y Mejoramiento de la Carretera ...,Concurso Público,...,NaN,NaN,NaN,None,None,None,None,None,None,0
3,2015918.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,2015918-1249-721938-2074791-45660,24.0,Consultoría de Obra,45660.0,Proyecto de Integración Vial Tacna-La Paz; Tra...,Concurso Público,...,NaN,NaN,NaN,None,None,None,None,None,None,0
4,2026767.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,2026767-16256-709096-2067605-143536,14.0,Consultoría de Obra,143536.0,CONSTRUCCIÓN DE VARIANTE EN EL SECTOR TUTUMBAR...,Concurso Público,...,NaN,NaN,NaN,None,None,None,None,None,None,0


2. Revisar columnas y detectar problemas

In [21]:
print("Columnas del dataset:")
df.columns.tolist()
df.dtypes


Columnas del dataset:


CODIGO_UNICO                      object
SECTOR                            object
DEPARTAMENTO                      object
NIVEL_GOBIERNO                    object
IDENTIFICADOR_OBRA                object
PROCESO                          float64
OBJETO_PROCESO                    object
CODIGO_OBRA                      float64
NOMBREOBRA                        object
METODO_CONTRATACION               object
TIEMPO_ABSOLUCION_CONSULTAS      float64
TIEMPO_PRESENTACION_OFERTAS      float64
__SOURCE                          object
CODIGO_RUC                       float64
RAZON_SOCIAL                      object
CONVOCATORIA_PROCESO_GANADO      float64
MIEMBROS_DE_COMITE                object
TOTALPROCESOSPARTICIPANTES       float64
CODIGO_CONTRATO                  float64
NUMERO_CONTRATO                   object
MONTO_CONTRACTUAL                float64
MONTO_REFERENCIAL                float64
MONTO_OFERTADO_PROMEDIO          float64
CONVOCATORIA                     float64
DNI_MIEMBRO_COMI

3. Renombrar la columna y_riesgo a riesgo si existe

In [22]:
if "y_riesgo" in df.columns:
    df.rename(columns={"y_riesgo": "riesgo"}, inplace=True)
    print("Columna renombrada: y_riesgo → riesgo")
else:
    print("Advertencia: No existe y_riesgo en este dataset.")


Columna renombrada: y_riesgo → riesgo


4. Limpiar valores nulos

In [23]:
print("Nulos antes:")
print(df.isnull().sum())

# Ejemplo: rellenar categóricas con "DESCONOCIDO"
categoricas = df.select_dtypes(include=["object"]).columns.tolist()
for col in categoricas:
    df[col] = df[col].fillna("DESCONOCIDO")

# Ejemplo: rellenar numéricas con mediana
numericas = df.select_dtypes(exclude=["object"]).columns.tolist()
for col in numericas:
    df[col] = df[col].fillna(df[col].median())

print("Nulos después:")
df.isnull().sum()


Nulos antes:
CODIGO_UNICO                         0
SECTOR                            6862
DEPARTAMENTO                      6862
NIVEL_GOBIERNO                    6862
IDENTIFICADOR_OBRA                5481
PROCESO                           6439
OBJETO_PROCESO                    8052
CODIGO_OBRA                       7494
NOMBREOBRA                        8052
METODO_CONTRATACION               8052
TIEMPO_ABSOLUCION_CONSULTAS       8052
TIEMPO_PRESENTACION_OFERTAS       8052
__SOURCE                             0
CODIGO_RUC                       13627
RAZON_SOCIAL                      8378
CONVOCATORIA_PROCESO_GANADO       8378
MIEMBROS_DE_COMITE               13723
TOTALPROCESOSPARTICIPANTES        8378
CODIGO_CONTRATO                   3129
NUMERO_CONTRATO                   3129
MONTO_CONTRACTUAL                13627
MONTO_REFERENCIAL                13627
MONTO_OFERTADO_PROMEDIO          13627
CONVOCATORIA                     12566
DNI_MIEMBRO_COMITE               12566
NOMBRE_MIEMB

CODIGO_UNICO                     0
SECTOR                           0
DEPARTAMENTO                     0
NIVEL_GOBIERNO                   0
IDENTIFICADOR_OBRA               0
PROCESO                          0
OBJETO_PROCESO                   0
CODIGO_OBRA                      0
NOMBREOBRA                       0
METODO_CONTRATACION              0
TIEMPO_ABSOLUCION_CONSULTAS      0
TIEMPO_PRESENTACION_OFERTAS      0
__SOURCE                         0
CODIGO_RUC                       0
RAZON_SOCIAL                     0
CONVOCATORIA_PROCESO_GANADO      0
MIEMBROS_DE_COMITE               0
TOTALPROCESOSPARTICIPANTES       0
CODIGO_CONTRATO                  0
NUMERO_CONTRATO                  0
MONTO_CONTRACTUAL                0
MONTO_REFERENCIAL                0
MONTO_OFERTADO_PROMEDIO          0
CONVOCATORIA                     0
DNI_MIEMBRO_COMITE               0
NOMBRE_MIEMBRO_COMITE            0
CODIGO_RUC_GANADOR               0
CODIGO_RUC_PARTICIPANTE          0
NOMBRE_PARTICIPANTE 

5. Limpiar columnas categóricas con valores incorrectos

In [24]:
for col in categoricas:
    print(f"\nValores únicos en {col}:")
    print(df[col].unique()[:20])



Valores únicos en CODIGO_UNICO:
['2002060.0' '2002210.0' '2015918.0' '2026767.0' '2027711.0' '2029683.0'
 '2031000.0' '2055893.0' '2055915.0' '2058698.0' '2058733.0' '2066003.0'
 '2077939.0' '2078482.0' '2080694.0' '2084815.0' '2090887.0' '2110320.0'
 '2110581.0' '2130855.0']

Valores únicos en SECTOR:
['TRANSPORTE' 'AGRARIA' 'INDUSTRIA; COMERCIO Y SERVICIOS'
 'SALUD Y SANEAMIENTO' 'OTROS' 'TRABAJO' 'COMUNICACIONES'
 'ADMINISTRACION Y PLANEAMIENTO' 'DEFENSA Y SEGURIDAD NACIONAL'
 'ENERGIA Y RECURSOS MINERALES' 'PESCA' 'AGROPECUARIA' 'SALUD'
 'PLANEAMIENTO; GESTIÓN Y RESERVA DE CONTINGENCIA'
 'ORDEN PÚBLICO Y SEGURIDAD' 'JUSTICIA' 'SANEAMIENTO' 'EDUCACIÓN'
 'ENERGÍA' 'DESCONOCIDO']

Valores únicos en DEPARTAMENTO:
['MULTIDEPARTAMENTAL' 'ANCASH' 'AREQUIPA' 'TUMBES' 'PUNO' 'HUANCAVELICA'
 'SAN MARTIN' 'LA LIBERTAD' 'LAMBAYEQUE' 'CUSCO' 'CAJAMARCA' 'PIURA'
 'AYACUCHO' 'MOQUEGUA' 'ICA' 'LIMA' 'HUANUCO' 'APURIMAC' 'TACNA' 'CALLAO']

Valores únicos en NIVEL_GOBIERNO:
['GOBIERNO NACIONAL' 'GO

In [25]:
for col in categoricas:
    df[col] = df[col].astype(str).str.strip().str.upper()


6. Eliminar columnas que NO deban ir al modelo

In [26]:
columnas_eliminar = [
    "descripcion", 
    "comentarios",
    "fecha_registro_original",
]

df.drop(columns=[c for c in columnas_eliminar if c in df.columns], inplace=True)

df.shape


(14179, 56)

7. Validar que la columna riesgo exista y sea binaria

In [27]:
if "riesgo" not in df.columns:
    raise RuntimeError("❌ ERROR: No existe la columna 'riesgo'. Revise el dataset.")
else:
    print("Columna de riesgo encontrada.")

print("Valores en riesgo:", df["riesgo"].unique())


Columna de riesgo encontrada.
Valores en riesgo: [0 1]


8. Guardar el dataset LIMPIO para modelado

In [28]:
df.to_parquet(DATASET_OUTPUT, index=False)
print("Dataset limpio guardado en:", DATASET_OUTPUT)


Dataset limpio guardado en: c:\IA_Investigacion\Deteccion_Corrupcion\data\processed\dataset_modelado.parquet


9. Limpieza EXPANDIDA de columnas complejas

In [36]:
# 1. Eliminar columnas de texto libre e identificadores inútiles
columnas_eliminar = [
    "IDENTIFICADOR_OBRA",
    "RAZON_SOCIAL",
    "MIEMBROS_DE_COMITE",
    "NUMERO_CONTRATO",
    "NOMBRE_MIEMBRO_COMITE",
    "NOMBRE_PARTICIPANTE",
    "NOMBRE_EMPRESA_GANADORA",
    "NOMBRE_EMPRESA_PARTICIPANTE",
    "NOMBRE_OBRA",
    "EMPRESA_EJECUTORA",
    "EMPRESA_SUPERVISORA",
    "RIESGO_DESCRIPCION_OBRA",
    "__SOURCE",
]

df.drop(columns=[c for c in columnas_eliminar if c in df.columns],
        inplace=True)

print("Columnas eliminadas:", [c for c in columnas_eliminar if c in df.columns])

# 2. Convertir fechas a timestamp numérico (si existen)
fechas_convertir = [
    "IND_FECHA_ADELANTO_MATERIALES",
    "IND_FECHA_ADELANTO_DIRECTO",
]

for col in fechas_convertir:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        df[col] = df[col].astype("int64") // 10**9  # convertir a epoch
        print(f"Columna fecha convertida a timestamp: {col}")

# 3. Intentar convertir algunas columnas a binarias (solo si realmente lo son)
columnas_binarias_candidatas = [
    "IND_INTERVENSION",
    "IND_RESIDENTE",
    "IND_MONTO_ADELANTO_MATERIALES",
    "IND_MONTO_ADELANTO_DIRECTO",
]

mapa_binario = {
    "SI": 1, "SÍ": 1, "TRUE": 1, "1": 1,
    "NO": 0, "FALSE": 0, "0": 0,
    "DESCONOCIDO": 0, "NAN": 0, "": 0,
    "NONE": 0, "NA": 0, "NO APLICA": 0,
}

columnas_binarias_final = []
columnas_no_binarias = []

for col in columnas_binarias_candidatas:
    if col not in df.columns:
        continue

    # Normalizar valores a mayúsculas y sin espacios
    serie_norm = (
        df[col]
        .astype(str)
        .str.strip()
        .str.upper()
    )

    valores_unicos = set(serie_norm.unique())
    print(f"\nValores únicos normalizados en {col}: {list(valores_unicos)[:20]}")

    # Verificar si todos los valores caben en el mapa binario (o son nulos/espacios)
    valores_validos = set(mapa_binario.keys())
    if all((v in valores_validos) for v in valores_unicos):
        # Se puede mapear como binaria
        df[col] = serie_norm.replace(mapa_binario).astype(int)
        columnas_binarias_final.append(col)
        print(f"✔ {col} convertida a binaria 0/1")
    else:
        # No es realmente binaria, la dejamos como categórica normal
        df[col] = serie_norm
        columnas_no_binarias.append(col)
        print(f"⚠ {col} NO es binaria pura, se mantiene como categórica.")

print("\nColumnas finalmente tratadas como binarias:", columnas_binarias_final)
print("Columnas candidatas que se dejaron como categóricas:", columnas_no_binarias)

# 4. Diagnóstico: columnas que NO se pueden convertir a float (por ahora informativo)
problemas = []
for col in df.columns:
    try:
        df[col].astype(float)
    except Exception:
        problemas.append(col)

print("\nColumnas aún no numéricas (se tratarán luego como categóricas en el modelo):")
print(problemas)


Columnas eliminadas: []
Columna fecha convertida a timestamp: IND_FECHA_ADELANTO_MATERIALES
Columna fecha convertida a timestamp: IND_FECHA_ADELANTO_DIRECTO

Valores únicos normalizados en IND_INTERVENSION: ['NORMAL', 'DESCONOCIDO']
⚠ IND_INTERVENSION NO es binaria pura, se mantiene como categórica.

Valores únicos normalizados en IND_RESIDENTE: ['ALERTA', 'NORMAL', 'DESCONOCIDO']
⚠ IND_RESIDENTE NO es binaria pura, se mantiene como categórica.

Valores únicos normalizados en IND_MONTO_ADELANTO_MATERIALES: ['ALERTA', 'NORMAL', 'DESCONOCIDO']
⚠ IND_MONTO_ADELANTO_MATERIALES NO es binaria pura, se mantiene como categórica.

Valores únicos normalizados en IND_MONTO_ADELANTO_DIRECTO: ['ALERTA', 'NORMAL', 'DESCONOCIDO']
⚠ IND_MONTO_ADELANTO_DIRECTO NO es binaria pura, se mantiene como categórica.

Columnas finalmente tratadas como binarias: []
Columnas candidatas que se dejaron como categóricas: ['IND_INTERVENSION', 'IND_RESIDENTE', 'IND_MONTO_ADELANTO_MATERIALES', 'IND_MONTO_ADELANTO_DIREC

10. Diagnóstico: ver qué valores tienen realmente las columnas "binarias"

In [37]:
# Diagnóstico para ver si realmente son binarias
for col in columnas_binarias_final:
    if col in df.columns:
        print(f"\nValores únicos de {col}:")
        print(df[col].unique())


11. Mapeo robusto SOLO DESPUÉS DEL DIAGNÓSTICO

In [38]:
mapa_binario = {
    "SI": 1, "SÍ": 1, "TRUE": 1, "1": 1,
    "NO": 0, "FALSE": 0, "0": 0,
    "DESCONOCIDO": 0, "NAN": 0, "": 0,
    "NONE": 0, "NA": 0, "NO APLICA": 0
}

for col in columnas_binarias_final:
    if col in df.columns:
        df[col] = (
            df[col].astype(str)
            .str.strip()
            .str.upper()
            .replace(mapa_binario)
        )
        # Todo lo que no quede en 1 se convierte automáticamente en 0
        df[col] = df[col].apply(lambda x: 1 if x == 1 else 0)


12. Validación final antes de guardar

In [39]:
print("✔ Validación final del dataset")

print("\nDimensiones:", df.shape)
print("Nulos totales:", df.isnull().sum().sum())

print("\nTipos:")
display(df.dtypes)

# Confirmación final
print("\nDataset listo para modelado.")


✔ Validación final del dataset

Dimensiones: (14179, 42)
Nulos totales: 0

Tipos:


CODIGO_UNICO                      object
SECTOR                            object
DEPARTAMENTO                      object
NIVEL_GOBIERNO                    object
PROCESO                          float64
OBJETO_PROCESO                    object
CODIGO_OBRA                      float64
METODO_CONTRATACION               object
TIEMPO_ABSOLUCION_CONSULTAS      float64
TIEMPO_PRESENTACION_OFERTAS      float64
CODIGO_RUC                       float64
CONVOCATORIA_PROCESO_GANADO      float64
TOTALPROCESOSPARTICIPANTES       float64
CODIGO_CONTRATO                  float64
MONTO_CONTRACTUAL                float64
MONTO_REFERENCIAL                float64
MONTO_OFERTADO_PROMEDIO          float64
CONVOCATORIA                     float64
DNI_MIEMBRO_COMITE               float64
CODIGO_RUC_GANADOR               float64
CODIGO_RUC_PARTICIPANTE          float64
RUC_GANADOR                      float64
RUC_PARTICIPANTE                 float64
MONTO_OFERTADO                   float64
ESTADO_OBRA     


Dataset listo para modelado.


Limpieza Final del Dataset

13.— Guardar versión final (sobrescribir la version del Notebook en Celda8)

In [40]:
df.to_parquet(DATASET_OUTPUT, index=False)
print("✔ Dataset FINAL guardado en:", DATASET_OUTPUT)


✔ Dataset FINAL guardado en: c:\IA_Investigacion\Deteccion_Corrupcion\data\processed\dataset_modelado.parquet
